# Chicken and egg in molecular metadynamics
Create protein folding landmarks from scratch



In [54]:
from pypdb import get_pdb_file

id="1L2Y"
pdb=get_pdb_file(id)

f=open(id + ".pdb","w")
f.write(pdb)
f.close()

In [55]:
import nglview as nv

v = nv.NGLWidget()
v.add_component(id + ".pdb")
v.clear()
v.add_representation('cartoon', selection='all', color='green')
v

NGLWidget()

In [67]:
import openbabel as ob
import math
from pyDOE import lhs
from pypdb import describe_pdb

pdb = id + ".pdb"
nres = int(describe_pdb(id)['nr_residues'])

out='conf%d.pdb'

# number of steps to twist a dihedral
nsteps = 12

# number of iterations; alltogether niter * nsteps random conformers are generated
#niter = 200
niter = 200

conf = 1
rgs = []

obc = ob.OBConversion()
obc.SetInAndOutFormats('pdb', 'pdb')
mol = ob.OBMol()
obc.ReadFile(mol,pdb)

# if I knew SMARTS better, oxygens would not have been counted
pattern = '[NH3]-[CH]-C(=O)'
for _ in range(nres-1):
    pattern += '-N-C-C(=O)'
    
sp = ob.OBSmartsPattern()
sp.Init(pattern)
sp.Match(mol)
match = sp.GetUMapList()

if len(match) != 1:
    print("backbone pattern not matched")
    raise
    
match = match[0]
    
def set_phi(mol,match,r,phi):
    a1 = mol.GetAtom(4*(r-1)-2)
    a2 = mol.GetAtom(4*(r-1))
    a3 = mol.GetAtom(4*(r-1)+1)
    a4 = mol.GetAtom(4*(r-1)+2)
    mol.SetTorsion(a1,a2,a4,a4,phi/180*math.pi)
    
def set_psi(mol,match,r,phi):
    a1 = mol.GetAtom(4*(r-1))
    a2 = mol.GetAtom(4*(r-1)+1)
    a3 = mol.GetAtom(4*(r-1)+2)
    a4 = mol.GetAtom(4*(r-1)+4)
    mol.SetTorsion(a1,a2,a4,a4,phi/180*math.pi)
    
def rg(mol):
    n = mol.NumAtoms()
    
#    x = [mol.GetAtom(i).GetCoordinate() for i in range(n)]
    for i in range(n):
        print(mol.GetAtom(i+1).GetCoordinate())
        
    mass = [mol.GetAtom(i).GetAtomicMass() for i in range(n)]
    xm = [(m*i,m*j,m*k) for (i,j,k),m in zip(x,mass)]
    tmass = sum(mass)
    rr = sum(mi*i+mj*j+mk*k for (i,j,k),(mi,mj,mk) in zip(x,xm))
    mm = sum((sum(i)/tmass)**2 for i in zip(*xm))
    return math.sqrt(rr/tmass - mm)

rgs = []

for _ in range(niter):
    phi = lhs(nres - 2, nsteps)
    psi = lhs(nres - 2, nsteps)

    for s in range(nsteps):
        for r in range(2,nres):
            set_phi(mol,match,r,phi[s][r-2] * 360)
            set_psi(mol,match,r,psi[s][r-2] * 360)
            
        fn = out % conf
        obc.WriteFile(mol,fn)
        
        rgs.append(rg(mol))
        conf += 1
            
print(rgs)


<Swig Object of type 'double *' at 0x7f39f85ed720>
<Swig Object of type 'double *' at 0x7f39f85eda20>
<Swig Object of type 'double *' at 0x7f39f85edae0>
<Swig Object of type 'double *' at 0x7f39f85edba0>
<Swig Object of type 'double *' at 0x7f39f85edc60>
<Swig Object of type 'double *' at 0x7f39f85edd20>
<Swig Object of type 'double *' at 0x7f39f85edde0>
<Swig Object of type 'double *' at 0x7f39f85edea0>
<Swig Object of type 'double *' at 0x7f39f85edf60>
<Swig Object of type 'double *' at 0x7f39f85cb060>
<Swig Object of type 'double *' at 0x7f39f85cb120>
<Swig Object of type 'double *' at 0x7f39f85cb1e0>
<Swig Object of type 'double *' at 0x7f39f85cb2a0>
<Swig Object of type 'double *' at 0x7f39f85cb360>
<Swig Object of type 'double *' at 0x7f39f85cb420>
<Swig Object of type 'double *' at 0x7f39f85cb4e0>
<Swig Object of type 'double *' at 0x7f39f85cb5a0>
<Swig Object of type 'double *' at 0x7f39f85cb660>
<Swig Object of type 'double *' at 0x7f39f85cb720>
<Swig Object of type 'double *'

AttributeError: 'NoneType' object has no attribute 'GetAtomicMass'

In [61]:
import nglview as nv

v = nv.show_url("conf1250.pdb")
v.clear()
v.add_representation('cartoon',selection='all',color='green')
v

NGLWidget()

In [ ]:
import math

